<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
os.chdir(f'./Transformer-Explainability')

!pip install -r requirements.txt
!pip install captum

fatal: destination path 'Transformer-Explainability' already exists and is not an empty directory.


In [2]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import (
    visualization
)
import torch

In [3]:
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2").to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["NEGATIVE", "POSITIVE"]


#Positive sentiment example

In [6]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('this', 0.4372084140777588), ('movie', 0.3218057453632355), ('was', 0.2817499041557312), ('the', 0.34847491979599), ('best', 0.6348710060119629), ('movie', 0.2984936833381653), ('i', 0.20119737088680267), ('have', 0.11754902452230453), ('ever', 0.1575811803340912), ('seen', 0.2046029269695282), ('!', 0.6018760204315186), ('some', 0.0220613032579422), ('scenes', 0.05102841556072235), ('were', 0.03648516908288002), ('ridiculous', 0.03679954633116722), (',', 0.01823556050658226), ('but', 0.4396149218082428), ('acting', 0.44878917932510376), ('was', 0.5093528032302856), ('great', 1.0), ('.', 0.03278274089097977), ('[SEP]', 0.10354921966791153)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


#Negative sentiment example

In [5]:
# encode a sentence
text_batch = ["I really didn't like this movie. Some of the actors were good, but overall the movie was boring."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.20068785548210144), ('really', -0.19849014282226562), ('didn', -0.29785579442977905), ("'", -0.01835285685956478), ('t', -0.37538254261016846), ('like', -0.16253846883773804), ('this', -0.19883419573307037), ('movie', -0.0518118217587471), ('.', -0.03121628239750862), ('some', -0.03469495102763176), ('of', -0.01185590960085392), ('the', -0.03453493490815163), ('actors', -0.10649579018354416), ('were', -0.025068022310733795), ('good', -0.043794937431812286), (',', -0.016170820221304893), ('but', -0.06994751840829849), ('overall', -0.3438393771648407), ('the', -0.2274700552225113), ('movie', -0.06602661311626434), ('was', -0.6292712092399597), ('boring', -1.0), ('.', -0.042594846338033676), ('[SEP]', -0.06377265602350235)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


# Choosing class for visualization example

In [8]:
# encode a sentence
text_batch = ["I love text visualizations."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# manually choose your classification here by the class index
classification = 1
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('i', 0.16306784749031067), ('love', 1.0), ('text', 0.09171640872955322), ('visual', 0.07082340121269226), ('##izations', 0.28751757740974426), ('.', 0.3706924319267273), ('[SEP]', 0.013394701294600964)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,[CLS] i love text visual ##izations . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,[CLS] i love text visual ##izations . [SEP]
